In [27]:
import numpy as np

In [70]:
data = "A gang of criminals rob a Gotham City mob bank, murdering each other until only the mastermind remains: the Joker, who escapes with the money. Batman, District Attorney Harvey Dent and Lieutenant Jim Gordon form an alliance to rid Gotham of organized crime. Bruce Wayne believes that, with Dent as Gotham's protector, he can retire from being Batman and lead a normal life with Rachel Dawes – even though she and Dent are dating. Mob bosses Sal Maroni, Gambol, and the Chechen hold a videoconference with their corrupt accountant, Lau, who has taken their funds for safekeeping and fled to Hong Kong. The Joker interrupts and warns them that Batman has no jurisdiction and is unhindered by the law, offering to kill him in exchange for half of their money. After Gambol puts a bounty on his head, the Joker kills Gambol and takes over his gang. The mob ultimately decides to take the Joker up on his offer. Batman finds Lau in Hong Kong and brings him back to Gotham to testify, allowing Dent to apprehend the entire mob. The Joker threatens to keep killing people unless Batman reveals his identity, and starts by murdering Police Commissioner Gillian B. Loeb and the judge presiding over the mob trial. The Joker also tries to kill Mayor Anthony Garcia, but Gordon sacrifices himself to stop the assassination. Dent learns that Rachel is the next target. Bruce decides to reveal his secret identity. Before he can, however, Dent falsely announces that he is Batman. Dent is taken into protective custody, but the Joker appears and attacks the convoy. Batman comes to Dent's rescue and Gordon, who faked his death, apprehends the Joker, securing a promotion to Commissioner. Rachel and Dent are escorted away by detectives on Maroni's payroll; Gordon later learns that they never arrived home. Batman interrogates the Joker, who reveals that they have been trapped in separate locations rigged with explosives. Batman races to save Rachel, while Gordon attempts to rescue Dent. Batman arrives at the building, but realizes that the Joker sent him to Dent's location instead. Both buildings explode, killing Rachel and disfiguring half of Dent's face. The Joker escapes with Lau, whom he later kills along with the Chechen. Coleman Reese, an accountant at Wayne Enterprises, deduces that Bruce is Batman and tries to go public with the information. Not wanting Reese's revelation to interfere with his plans, the Joker threatens to destroy a hospital unless someone kills Reese within an hour. Gordon orders the evacuation of all the hospitals in Gotham and goes to secure Reese. The Joker gives Dent a gun and convinces him to seek revenge for Rachel's death, then destroys the hospital and escapes with a busload of hostages. Dent goes on a killing spree, deciding the fates of people he holds responsible for Rachel's death by flipping his lucky coin. After announcing Gotham will be subject to his rule come nightfall, the Joker rigs two evacuating ferries with explosives; one containing civilians and the other containing prisoners. He says that he will blow them both up by midnight, but will let one live if its passengers (who have been supplied the trigger to the other boat's explosives) blow up the other. Batman finds the Joker by using a sonar device that spies on the entire city, with the reluctant help of Lucius Fox. Both the civilians and the prisoners refuse to kill each other, while Batman apprehends the Joker after a brief fight. Before the police arrive to take the Joker into custody, he gloats that Gotham's citizens will lose hope once Dent's rampage becomes public knowledge. Gordon and Batman arrive at the building where Rachel perished. Dent shoots Batman, spares himself, and threatens to kill Gordon's son, claiming that Gordon's negligence is responsible for Rachel's death. Before he can flip for the boy, Batman, who was wearing body armor, tackles Dent off the building to his death. Batman persuades Gordon to hold him responsible for the killing spree to preserve Dent's heroic image. As the police launch a manhunt for Batman, Gordon destroys the Bat-signal, Fox watches as the sonar device self-destructs, and Alfred Pennyworth burns a letter from Rachel saying she planned to marry Dent. Before the release of Batman Begins, screenwriter David S. Goyer wrote a treatment for two sequels which introduced the Joker and Harvey Dent. His original intent was for the Joker to scar Dent during the Joker's trial in the third film, turning Dent into Two-Face. Goyer, who penned the first draft of the film, cited the DC Comics 13-issue comic book limited series Batman: The Long Halloween as the major influence on his storyline. According to veteran Batman artist Neal Adams, he met with David Goyer in Los Angeles, and the story would eventually look to Adams and writer Denny O'Neil's 1971 story The Joker's Five-Way Revenge that appeared in Batman #251, in which O'Neil and Adams re-introduced the Joker.[61] While initially uncertain of whether or not he would return to direct the sequel, Nolan did want to reinterpret the Joker on screen. On July 31, 2006, Warner Bros. officially announced initiation of production for the sequel to Batman Begins titled The Dark Knight;[62] it is the first live-action Batman film without the word  Batman  in its title, which Bale noted as signaling that  this take on Batman of mine and Chris' is very different from any of the others . After much research, Nolan's brother and co-writer, Jonathan, suggested the Joker's first two appearances, published in the first issue of Batman (1940), as the crucial influences. Christopher had Jonathan watch Fritz Lang's 1933 crime film The Testament of Dr. Mabuse prior to writing the Joker, with the Joker resembling Mabuse's characteristics. Christopher Nolan referred to Lang's film as  essential research for anyone attempting to write a supervillain . Jerry Robinson, one of the Joker's co-creators, was consulted on the character's portrayal.[68] Nolan decided to avoid divulging an in-depth origin story for the Joker, and instead portray his rise to power so as to not diminish the threat he poses, explaining to MTV News,  the Joker we meet in The Dark Knight is fully formed ... To me, the Joker is an absolute. There are no shades of gray to him – maybe shades of purple. He's unbelievably dark. He bursts in just as he did in the comics.  Nolan reiterated to IGN,  We never wanted to do an origin story for the Joker in this film , because  the arc of the story is much more Harvey Dent's; the Joker is presented as an absolute. It's a very thrilling element in the film, and a very important element, but we wanted to deal with the rise of the Joker, not the origin of the Joker.  Nolan suggested Batman: The Killing Joke influenced a section of the Joker's dialogue in the film, in which he says that anyone can become like him given the right circumstances.[70] Nolan also cited Heat as  sort of an inspiration  for his aim  to tell a very large, city story or the story of a city :  If you want to take on Gotham, you want to give Gotham a kind of weight and breadth and depth in there. So you wind up dealing with the political figures, the media figures. That's part of the whole fabric of how a city is bound together . According to Nolan, an important theme of the sequel is  escalation , extending the ending of Batman Begins, noting  things having to get worse before they get better . While indicating The Dark Knight would continue the themes of Batman Begins, including justice vs. revenge and Bruce Wayne's issues with his father, Nolan emphasized the sequel would also portray Wayne more as a detective, an aspect of his character not fully developed in Batman Begins. Nolan described the friendly rivalry between Bruce Wayne and Harvey Dent as the  backbone  of the film. He also chose to compress the overall storyline, allowing Dent to become Two-Face in The Dark Knight, thus giving the film an emotional arc the unsympathetic Joker could not offer. Nolan acknowledged the title was not only a reference to Batman, but also the fallen  white knight  Harvey Dent."
chars = list(set(data))
print(chars)
data_size, vocab_size = len(data), len(chars)
# print(data_size)
# print(vocab_size)
print("data has "+str(data_size)+" characters, "+str(vocab_size)+" unique.")

char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }
print(char_to_ix)
print(ix_to_char)

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

['8', 'j', 'z', 'I', 'v', '–', ')', 'B', 'H', 'F', '9', '#', 'o', 'a', ':', 'W', '[', '0', 'd', ' ', 'O', 'r', 'D', 'J', 'E', '5', 'M', '4', 'p', 'b', 'g', 'u', 't', 'q', 'V', 'e', 'k', 'K', 'P', 'G', 'm', ']', '-', 's', 'x', 'N', 'n', 'l', 'C', 'T', ',', 'L', 'i', 'f', '.', ';', 'S', "'", '2', 'h', 'c', '1', '6', '(', 'y', '7', 'R', 'w', 'A', '3']
data has 8116 characters, 70 unique.
{'8': 0, ';': 55, 'j': 1, 'c': 60, 'z': 2, '6': 62, 'I': 3, 'v': 4, '–': 5, ')': 6, ' ': 19, 'F': 9, 'H': 8, '9': 10, '#': 11, 'r': 21, 'o': 12, 'a': 13, ':': 14, 'W': 15, '0': 17, 'd': 18, '5': 25, 'O': 20, 'D': 22, 'E': 24, 'J': 23, 'B': 7, 'M': 26, '4': 27, 'p': 28, 'g': 30, 'f': 53, 'q': 33, 't': 32, 'e': 35, 'k': 36, 'K': 37, 'u': 31, 'P': 38, 'G': 39, 'm': 40, ']': 41, '-': 42, 's': 43, 'x': 44, 'N': 45, 'n': 46, 'R': 66, 'l': 47, 'C': 48, 'T': 49, 'b': 29, 'L': 51, 'i': 52, '.': 54, 'S': 56, "'": 57, 'h': 59, '[': 16, '1': 61, '(': 63, 'V': 34, '2': 58, 'y': 64, '7': 65, ',': 50, 'w': 67, 'A': 68, 

In [71]:
def array2text(arr):
    text=""
    for i in arr:
        text+=ix_to_char[i]
    return text

print(array2text([2, 4, 16, 21, 4, 25]))
def text2array(string):
    arr=[]
    for i in string:
        arr.append(char_to_ix[i])
    return arr

print(text2array("Batman"))

zv[rv5
[7, 13, 32, 40, 13, 46]


In [ ]:
def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  if len(inputs)!=len(targets):
      targets.append(0)
#   print(len(inputs))
#   print(len(targets))
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
#     print("ps")
#     print(ps[t])
#     print("target")
#     print(targets[t])
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while smooth_loss>=0.005:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
#   print(array2text(inputs)+" with length "+str(len(inputs)))
#   print(array2text(targets)+" with length "+str(len(targets)))
  # sample from the model now and then
#   if n % 100 == 0:
#     sample_ix = sample(hprev, inputs[0], 200)
#     txt = ''.join(ix_to_char[ix] for ix in sample_ix)
#     print("----\n "+txt+" \n----")

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print("iter "+str(n)+", loss: "+str(smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

iter 0, loss: 106.14736424814065
iter 100, loss: 105.09035661338844
iter 200, loss: 101.98857543544513
iter 300, loss: 98.99517212266446
iter 400, loss: 95.97775253759515
iter 500, loss: 93.07922164082
iter 600, loss: 90.59928673683659
iter 700, loss: 88.1830286772209
iter 800, loss: 85.72677095115213
iter 900, loss: 83.74674636590389
iter 1000, loss: 81.78151786243228
iter 1100, loss: 79.8547515382697
iter 1200, loss: 78.29539871059919
iter 1300, loss: 76.89018106344525
iter 1400, loss: 75.32021072869374
iter 1500, loss: 73.94967418109951
iter 1600, loss: 72.80736439071204
iter 1700, loss: 71.58980434879604
iter 1800, loss: 70.2926110136401
iter 1900, loss: 69.36075240180249
iter 2000, loss: 68.46601955823522
iter 2100, loss: 67.28047860602457
iter 2200, loss: 66.51909090693044
iter 2300, loss: 65.76473937862374
iter 2400, loss: 64.81702500933618
iter 2500, loss: 64.08364981972697
iter 2600, loss: 63.50004410805592
iter 2700, loss: 62.73192533211291
iter 2800, loss: 62.0797632583403
i

iter 23000, loss: 40.75635567055962
iter 23100, loss: 40.73782095309175
iter 23200, loss: 40.76079497806088
iter 23300, loss: 40.694546485688726
iter 23400, loss: 40.75458869093272
iter 23500, loss: 40.70659535101307
iter 23600, loss: 40.494702757019546
iter 23700, loss: 40.60378154643465
iter 23800, loss: 40.5176352704826
iter 23900, loss: 40.382864303765416
iter 24000, loss: 40.48609412739141
iter 24100, loss: 40.485187828883625
iter 24200, loss: 40.3033792080996
iter 24300, loss: 40.29731557260831
iter 24400, loss: 40.289857195549054
iter 24500, loss: 40.2261995721751
iter 24600, loss: 40.110337171675276


In [34]:
def predict(inputs,hprev):
#   print(inputs)
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  predicted=-1
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t])
    predicted = np.argmax(ps[t])
#     print(predicted)
  print(ix_to_char[predicted])

In [68]:
input = "Darkestart" ##Batman ##[18,37,34,36] ##Nola
hprev = np.zeros((hidden_size,1))
predict(text2array(input),hprev)